In [1]:
import os
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup as bs
import time
from difflib import get_close_matches
from fuzzywuzzy import process

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [157]:
leaguesDic = {
   "Argentinian-Liga-A": "Liga_Nacional_de_B%C3%A1squet",
    "Basketball-Champions-League-": "Basketball_Champions_League",
    "Adriatic-Junior-Liga-ABA": "Junior_ABA_League",
    "Spanish-LEB-Gold": "LEB_Oro",
    "Italian-Cup": "Italian_Basketball_Cup",
    "French-Leaders-Cup-LNB-B": "LNB_Pro_B_Leaders_Cup",
    "Czech-NBL": "National_Basketball_League_(Czech_Republic)",
    "Austrian-A-Bundesliga": "Austrian_Basketball_Bundesliga",
    "Alpe-Adria-Cup": "Alpe_Adria_Cup",
    "Serbian-KLS": "Basketball_League_of_Serbia",
    "Spanish-Cup": "Copa_del_Rey_de_Baloncesto",
    "BNXT-League": "BNXT_League",
    "FIBA-Europe-Cup---Qualification": "FIBA_Europe_Cup",
    "German-Pro-A": "ProA",
    "Greek-HEBA-A1": "Greek_Basket_League",
    "Cypriot-Division-A": "Cypriot_Basketball_Division_A",
    "Basketball-Africa-League": "Basketball_Africa_League",
    "Georgian-Super-Liga": "Georgian_Superliga",
    "Bulgarian-NBL": "Bulgarian_Basketball_League",
    "NBL-Blitz": "NBL_Blitz",
    "Portuguese-LPB": "Liga_Portuguesa_de_Basquetebol",
    "NBA": "NBA",
    "Swiss-LNA": "Swiss_Basketball_League",
    "British-BBL": "British_Basketball_League",
    "Mexican-CIBACOPA": "CIBACOPA",
    "Lithuanian-NKL": "National_Basketball_League_(Lithuania)",
    "Belgrade-Tournament-(ANGT)": "Adidas_Next_Generation_Tournament",
    "Slovakian-Extraliga": "Slovak_Basketball_League",
    "Romanian-Divizia-A": "Liga_Na%C8%9Bional%C4%83_(men%27s_basketball)",
    "Munich-Tournament-(ANGT)": "Adidas_Next_Generation_Tournament",
    "Lebanese-Division-A": "Lebanese_Basketball_League",
    "VTB-Youth-United-League": "VTB_United_Youth_League",
    "French-Jeep-Elite": "LNB_Pro_A",
    "Norwegian-BLNO": "BLNO",
    "Montenegrin-Prva-A-Liga": "Prva_A_Liga",
    "VTB-United-League": "VTB_United_League",
    "French-NM1": "LNB_Pro_B",
    "New-Zealand-NB": "National_Basketball_League_(New_Zealand)",
    "Luxembourg-Total-League": "Total_League",
    "Turkish-TBL": "Basketball_Super_League",
    "German-BBL": "Basketball_Bundesliga",
    "Chinese-CBA": "Chinese_Basketball_Association",
    "Puerto-Rican-BSN": "Baloncesto_Superior_Nacional",
    "Balkan-BIBL": "BIBL",
    "Bosnian-BiH-Liga": "Basketball_Championship_of_Bosnia_and_Herzegovina",
    "French-Leaders-Cup-LNB-A": "LNB_Pro_A_Leaders_Cup",
    "Swedish-Basketligan": "Basketligan",
    "Basketball-Africa-League---Qualification": "Basketball_Africa_League",
    "FIBA-Europe-Cup": "FIBA_Europe_Cup",
    "Japanese-B.League": "B.League",
    "Turkish-BSL": "Basketball_Super_League",
    "German-Pro-B": "ProB",
    "Euroleague": "EuroLeague",
    "Italian-Serie-A2-Basket": "Serie_A2_(basketball)",
    "Greek-HEBA-A2": "Greek_Basketball_League",
    "Canadian-NBL": "National_Basketball_League_of_Canada",
    "French-LNB-Pro-B": "LNB_Pro_B",
    "PBA---Philippine-Cup": "PBA_Philippine_Cup",
    "Eurocup": "EuroCup_Basketball",
    "Brazilian-NBB": "Novo_Basquete_Brasil",
    "Belarusian-BPL": "Belarusian_Premier_League_(basketball)",
    "Ukrainian-Superleague": "Ukrainian_Basketball_SuperLeague",
    "Turkish-Cup": "Turkish_Basketball_Cup",
    "Danish-Basketligaen": "Basketligaen",
    "PBA---Governors-Cup": "PBA_Governors'_Cup",
    "Intercontinental-Cup": "FIBA_Intercontinental_Cup",
    "Croatian-A-1-Liga": "Croatian_Basketball_League",
    "Venezuelan-SLB": "Superliga_Profesional_de_Baloncesto",
    "Patras-Tournament-(ANGT)": "Adidas_Next_Generation_Tournament",
    "PBA---Commissioners-Cup": "PBA_Commissioner's_Cup",
    "Estonian-Latvian-Basketball-League": "Baltic_Basketball_League",
    "Slovenian-SKL": "Slovenian_Basketball_League",
    "Serbian-RK-Cup": "Radivoj_Korac_Cup",
    "Lithuanian-LKL": "Lietuvos_krep%C5%A1inio_lyga",
    "Australian-NBL": "National_Basketball_League_(Australia)",
    "Spanish-LEB-Silver": "LEB_Plata",
    "Italian-Lega-Basket-Serie-A": "Lega_Basket_Serie_A",
    "French-LNB-Espoirs": "LNB_Pro_A",
    "Macedonian-Superleague": "Macedonian_First_League_(basketball)",
    "Spanish-ACB-SuperCup": "Supercopa_de_España_de_Baloncesto",
    "Hungarian-NBIA": "NBIA",
    "Israeli-BSL": "Israeli_Basketball_Premier_League",
    "VTB-SuperCup": "VTB_United_League",
    "Finnish-Korisliiga": "Korisliiga",
    "Mexican-LNBP": "Liga_Nacional_de_Baloncesto_Profesional",
    "Kosovo-FBK": "Kosovo_Basketball_Superleague",
    "Spanish-ACB": "Liga_ACB",
    "Canadian-Elite-Basketball-League": "Canadian_Elite_Basketball_League",
    "German-Cup": "BBL-Pokal",
    "Polish-EB": "Polish_Basketball_League",
    "South-Korean-KBL": "Korean_Basketball_League",
    "Adriatic-League-Liga-ABA": "ABA_League",
    "Youth-Basketball-Champions-League": "Basketball_Champions_League",
    "Zadar-Tournament-(ANGT)": "Adidas_Next_Generation_Tournament"
}
dynamicURL  = "https://en.wikipedia.org/wiki/"
dfList =[]


In [138]:
#Checking that all URLs are valid
for league in leaguesDic.values():
        res = req.get(dynamicURL+league)
        if(res.status_code == 404):
            print(f"{res} in {league}")
print("finish")

<Response [404]> in NBL_Blitz
finish


In [135]:
##Inserting All Tables from Wiki to a dictionry with League Name as a key
tablesDic = {}
count =0
for league in leaguesDic:
    currURL = dynamicURL + leaguesDic[league]
    try:
        tablesDic[league] = pd.read_html(currURL)
        print(f"for {league} tables has been added!")
    except:
        tablesDic[league] = None
        count+=1
        print(f"for {league} inseration has failed!")
        continue
print(f"total Failed:{count}")

for Argentinian-Liga-A tables has been added!
for Basketball-Champions-League- tables has been added!
for  Adriatic-Junior-Liga-ABA tables has been added!
for Spanish-LEB-Gold tables has been added!
for Italian-Cup tables has been added!
for French-Leaders-Cup-LNB-B tables has been added!
for Czech-NBL tables has been added!
for Austrian-A-Bundesliga tables has been added!
for Alpe-Adria-Cup tables has been added!
for  Serbian-KLS tables has been added!
for Spanish-Cup tables has been added!
for BNXT-League tables has been added!
for FIBA-Europe-Cup---Qualification tables has been added!
for German-Pro-A inseration has failed!
for  Greek-HEBA-A1 tables has been added!
for Cypriot-Division-A tables has been added!
for Basketball-Africa-League tables has been added!
for Georgian-Super-Liga tables has been added!
for Bulgarian-NBL tables has been added!
for NBL-Blitz inseration has failed!
for Portuguese-LPB tables has been added!
for NBA tables has been added!
for Swiss-LNA tables has be

In [136]:
##Taking out all the Champions Tables to new dictonary From allTables dic
## Need To take Out Venzuelian,ProA,BLitz
championsTableDic={}
counter=0
for leagueName in tablesDic:
    print(leagueName)
    if(tablesDic[leagueName] != None):
        for table in tablesDic[leagueName]:
            if(type(table.columns) == pd.MultiIndex): ## Checking if the word is in MultiIndex
                if("Champion" in table.columns.get_level_values(1) or "Champions" in table.columns.get_level_values(1) or "Champions" in table.columns.get_level_values(0) or "Champion" in table.columns.get_level_values(0) or "Gold" in table.columns.get_level_values(1)):
                    counter +=1
                    championsTableDic[leagueName] = table
                    print(f"The Champions table has been found for:{leagueName}")
                    break
                
                
            elif("Champion" in table.columns or "Winners" in table.columns or "Champions" in table.columns or "Years won" in table.columns or "Year(s) won" in table.columns or "Winner" in table.columns or "Champion years" in table.columns):
                counter +=1
                championsTableDic[leagueName] = table
                print(f"The Champions table has been found for:{leagueName}")
                break
            
        
                
print(len(tablesDic))
print(counter)

Argentinian-Liga-A
The Champions table has been found for:Argentinian-Liga-A
Basketball-Champions-League-
The Champions table has been found for:Basketball-Champions-League-
 Adriatic-Junior-Liga-ABA
The Champions table has been found for: Adriatic-Junior-Liga-ABA
Spanish-LEB-Gold
The Champions table has been found for:Spanish-LEB-Gold
Italian-Cup
The Champions table has been found for:Italian-Cup
French-Leaders-Cup-LNB-B
The Champions table has been found for:French-Leaders-Cup-LNB-B
Czech-NBL
The Champions table has been found for:Czech-NBL
Austrian-A-Bundesliga
The Champions table has been found for:Austrian-A-Bundesliga
Alpe-Adria-Cup
The Champions table has been found for:Alpe-Adria-Cup
 Serbian-KLS
The Champions table has been found for: Serbian-KLS
Spanish-Cup
The Champions table has been found for:Spanish-Cup
BNXT-League
The Champions table has been found for:BNXT-League
FIBA-Europe-Cup---Qualification
The Champions table has been found for:FIBA-Europe-Cup---Qualification
Germa

In [154]:
##Saving all Tables to CSV
folderPath = os.getcwd()+ os.sep +"Champions_Tables"
print(folderPath)
for leauge in championsTableDic:
    print(type(championsTableDic[leauge]))
    df = championsTableDic[leauge]
    ##Before Saving Make all the Tables in Same Format of Needed Cols
    
    df.to_csv(folderPath+"/"+leauge+".csv",index=False)
 

/Users/araddavid/Desktop/Project Python/Predict-Playoff-Teams/Champions_Tables
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'p

In [8]:
##Start Running From Here####
dfListSeason=[]
dfListClub=[]
folderPath = os.getcwd()+ os.sep +"Champions_Tables"

championsTables =  os.listdir(folderPath)
championsTables.remove(".DS_Store")
###Taking out all the tables and ordring it into 2 diffrent lists
for table in championsTables:
    leagueName = table[0:-4]
    df = pd.read_csv(folderPath+"/"+table)
    if("Season" in df.columns or "Year" in df.columns):
        df = df.rename(columns={"Year":"Season"}) ## Making all Dfs Simmilar
        dfListSeason.append((leagueName,df))
    else:
        df = df.rename(columns={"Club":"Team","Teams":"Team","Champion":"Team","Champions":"Team"})
        dfListClub.append((leagueName,df))
       

In [9]:
countAtte=0
champTuppleYears={}
##Moving on the tables that are by Season as row and adding the final label
for curr in dfListSeason:
    lName = curr[0]
    spefTable = curr[1]
    spefFolderPath = os.getcwd()+ os.sep +lName
    
    for col in ["Champions","Champion","Winner","Winners","Final","Finals"]:
        try:
            champTuppleYears[lName]= (list(zip(spefTable["Season"],spefTable[col])))
            countAtte=0
            break
            
        except:
            countAtte+=1
            if(countAtte == 6):
                print(f"Dosent Work for this Leauge:{lName}")
            continue
            
    
  
    

In [10]:
champTuppleClub={}
##Moving on the tables that are by Team as row and adding the final label
for curr in dfListClub:
    name=curr[0]
    clubTable = curr[1]
    spefFolderPath = os.getcwd()+ os.sep +name

    for col in ["Winning Years","Year(s) won","Years won","Years","league title","Winning years"]:
        try:
            champTuppleClub[name]= (list(zip(clubTable["Team"],clubTable[col])))
        except:
            continue
    

In [11]:
####Here need to build the function to fit the lables to the tables
##Handle The Years
totCounter=0
succCounter=0
for leauge in champTuppleYears:
    for season in champTuppleYears[leauge]:
        totCounter+=1
        ###Taking Out the Year and champ
        currSeason = season[0]
        currChamp = season[1]
           
        try:
            ##The Years in the tables format is : 2022-23 for 2023 season ,so taking the right side and doing as normal year Presnation
            if(int(season[0][-2:len(season[0])]) <= 23):
                currSeason = 2000 + int(season[0][-2:len(season[0])])
            else:
                currSeason = 1900 + int(season[0][-2:len(season[0])])
                
            ###Loading The Correct Table  
            if(leauge == "Zadar-Tournament-(ANGT)" or leauge == "Youth-Basketball-Champions-League"): ##Those 2 are not in Folders 
                currTableLink = os.getcwd()+ os.sep + leauge + "_" + str(currSeason)+".csv"
            else:
                 currTableLink = os.getcwd()+ os.sep + leauge + os.sep + leauge + "_" + str(currSeason)+".csv"
                
            currTable = pd.read_csv(currTableLink)
            ##Last CheckPoint
            print(f"In:{leauge} Season:{currSeason} the champion is:{currChamp}")
            
            ##Here We need to add function thats finds team in Team Colunm that close enougth to currChamp 
            #champInTable = get_close_matches(currChamp,currTable["Team"])[0]
            champInTable = process.extractOne(currChamp, currTable["Team"])
            print(champInTable[0])
            if(len(champInTable) >=1):
                succCounter+=1

        
            mask = currTable["Team"].str.contains(champInTable[0])
            currTable["is_Champion"] = np.where(mask,1,0)
            
            ###Saving the Tables with the labels
            currTable.to_csv(currTableLink,index=False)
               
        except:
               continue
                
print(f"Succses for:{succCounter} from:{totCounter}")             
            
                
                
                
     
              
                
            
                
             
        
            
     

In:Romanian-Divizia-A Season:2012 the champion is:Asesoft Ploiești
CSU Asesoft Ploiesti
In:Romanian-Divizia-A Season:2013 the champion is:Asesoft Ploiești
CSU Asesoft Ploiesti
In:Romanian-Divizia-A Season:2014 the champion is:Asesoft Ploiești
CSU Asesoft Ploiesti
In:Romanian-Divizia-A Season:2015 the champion is:Asesoft Ploiești
CSU Asesoft Ploiesti
In:Romanian-Divizia-A Season:2016 the champion is:CSM Oradea
CSM Oradea
In:Romanian-Divizia-A Season:2017 the champion is:U-BT Cluj-Napoca
U-Banca Transilvania Cluj
In:Romanian-Divizia-A Season:2018 the champion is:CSM Oradea
CSM Oradea
In:Romanian-Divizia-A Season:2019 the champion is:CSM Oradea
CSM Oradea
In:Romanian-Divizia-A Season:2020 the champion is:Cancelled due to the COVID-19 pandemic
BC Athletic Constanta
In:Romanian-Divizia-A Season:2021 the champion is:U-BT Cluj-Napoca
U-Banca Transilvania Cluj
In:Romanian-Divizia-A Season:2022 the champion is:U-BT Cluj-Napoca
U-Banca Transilvania Cluj
In:VTB-Youth-United-League Season:2018 the

In:Spanish-ACB Season:2018 the champion is:Real Madrid
Real Madrid
In:Spanish-ACB Season:2019 the champion is:Real Madrid
Real Madrid
In:Spanish-ACB Season:2021 the champion is:Barça
Barca
In:Spanish-ACB Season:2022 the champion is:Real Madrid
Real Madrid
In:Danish-Basketligaen Season:2022 the champion is:Bakken Bears (20)
Bakken Bears
In:Danish-Basketligaen Season:2021 the champion is:Bakken Bears (19)
Bakken Bears
In:Danish-Basketligaen Season:2020 the champion is:Bakken Bears (18)
Bakken Bears
In:Danish-Basketligaen Season:2019 the champion is:Bakken Bears (17)
Bakken Bears
In:Danish-Basketligaen Season:2018 the champion is:Bakken Bears (16)
Bakken Bears
In:Danish-Basketligaen Season:2017 the champion is:Bakken Bears (15)
Bakken Bears
In:Danish-Basketligaen Season:2016 the champion is:Horsens IC (6)
Horsens IC
In:Danish-Basketligaen Season:2015 the champion is:Horsens IC (5)
Horsens IC
In:Danish-Basketligaen Season:2014 the champion is:Bakken Bears (15)
Bakken Bears
In:Danish-Basket

/var/folders/gt/bb_0dv113qj2plqgwdz8dxhc0000gn/T/ipykernel_18296/1696700337.py:37: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = currTable["Team"].str.contains(champInTable[0])


In:French-LNB-Pro-B Season:2018 the champion is:ADA Blois
ADA Blois Basket
In:French-LNB-Pro-B Season:2019 the champion is:Roanne
Chorale Roanne Basket
In:French-LNB-Pro-B Season:2020 the champion is:Canceled due to the COVID-19 pandemic
Antibes
In:French-LNB-Pro-B Season:2021 the champion is:Fos
Fos Sur Mer
In:French-LNB-Pro-B Season:2022 the champion is:SLUC Nancy (2)
Nancy Basket
In:Lithuanian-NKL Season:2018 the champion is:Neptūnas-Akvaservis Klaipėda[23]
Neptuans Akvaservis Klaipeda
In:Lithuanian-NKL Season:2019 the champion is:Sūduva-Mantinga Marijampolė[25]
Marijampole Suduva-Arvi
In:Lithuanian-NKL Season:2020 the champion is:Sūduva-Mantinga Marijampolė[27]
Marijampole Suduva-Arvi
In:Lithuanian-NKL Season:2021 the champion is:Cbet Jonava[28]
Jonava Jonavos
In:Lithuanian-NKL Season:2022 the champion is:Gargždai[30]
Gargzdu Gargzdai
In:Lithuanian-NKL Season:2023 the champion is:Žalgiris-2 Kaunas[32]
Zalgris II
In:Italian-Serie-A2-Basket Season:2012 the champion is:Trenkwalder Reg

In:French-NM1 Season:2022 the champion is:SLUC Nancy (2)
Besancon BCD
In:Norwegian-BLNO Season:2012 the champion is:Frøya
Froya Basket
In:Norwegian-BLNO Season:2013 the champion is:Bærum
Baerum Basket
In:Norwegian-BLNO Season:2014 the champion is:Gimle
Gimle BBK
In:Norwegian-BLNO Season:2015 the champion is:Asker Aliens
Asker Aliens
In:Norwegian-BLNO Season:2016 the champion is:Centrum Tigers
Centrum Tigers
In:Norwegian-BLNO Season:2017 the champion is:Centrum Tigers
Centrum Tigers
In:Norwegian-BLNO Season:2018 the champion is:Kongsberg Miners
BK Kongsberg Miners
In:Norwegian-BLNO Season:2019 the champion is:Kongsberg Miners
BK Kongsberg Miners
In:Norwegian-BLNO Season:2020 the champion is:Cancelled due to the COVID-19 pandemic
Baerum Basket
In:Norwegian-BLNO Season:2021 the champion is:Cancelled due to the COVID-19 pandemic
Baerum Basket
In:Norwegian-BLNO Season:2022 the champion is:Gimle
Gimle BBK
In:Slovenian-SKL Season:2012 the champion is:Krka
Krka Novo Mesto
In:Slovenian-SKL Seas

In:Japanese-B.League Season:2019 the champion is:Alvark Tokyo (4)
Toyota Alvark
In:Japanese-B.League Season:2020 the champion is:Cancelled due to the COVID-19 pandemic
Shimane Susanoo Magic
In:Japanese-B.League Season:2021 the champion is:Chiba Jets Funabashi (2)
Chiba Jets
In:Japanese-B.League Season:2022 the champion is:Utsunomiya Brex (2)
Link Tochigi Brex
In:Japanese-B.League Season:2023 the champion is:TBD
Chiba Jets
In:Belgrade-Tournament-(ANGT) Season:2012 the champion is:Lietuvos rytas
INSEP
In:Belgrade-Tournament-(ANGT) Season:2013 the champion is:FIATC Joventut
KK Crvena Zvezda Youth
In:Belgrade-Tournament-(ANGT) Season:2014 the champion is:Crvena zvezda Telekom
KK Crvena Zvezda Youth
In:Belgrade-Tournament-(ANGT) Season:2015 the champion is:Real Madrid
KK Crvena Zvezda Youth
In:Belgrade-Tournament-(ANGT) Season:2016 the champion is:Barcelona Lassa
ALBA Berlin U18
In:Belgrade-Tournament-(ANGT) Season:2017 the champion is:Centre Fédéral
Better Academy Prague
In:Belgrade-Tourna

In [12]:

dontNeedYearsManip = ["NBA","Youth-Basketball-Champions-League","Bulgarian-NBL","Macedonian-Superleague","Turkish-BSL","Basketball-Champions-League-","Bosnian-BiH-Liga"
                     ,"Czech-NBL","Basketball-Africa-League","New-Zealand-NB","Spanish-Cup","Australian-NBL","Turkish-TBL","French-Leaders-Cup-LNB-A",
                     "FIBA-Europe-Cup---Qualification","Puerto-Rican-BSN","Basketball-Africa-League---Qualification","FIBA-Europe-Cup","Mexican-CIBACOPA"]

for league in champTuppleClub:

    print(league)
    ##Passing all the Teams and their years :  
    for teamDet in champTuppleClub[league]:
        team = teamDet[0]
        years = str(teamDet[1]).split(",") ## need to split the years by , beacuse its format
     
            
        for i in years: ## Passing trough all teams champioship Years
            if(league not in dontNeedYearsManip): ## some Tables their format is :1944-45 so need to take out the 45 and make it as 1945 ...
                try:
                    currYear = i.split("–")[1]
                    ## all Year that <= 23 it's 2000+ and >23 its 1900
                    if(int(currYear) <=23):
                        currYear = str(2000 + int(currYear))
                    else:
                        currYear = str(1900 + int(currYear))
                except:
                    currYear = i
            else:
                currYear = i
            ##Some years was saved with spacebar at the start so need to take it off  
            if(currYear[0] == " "):
                 currYear = currYear[1:len(currYear)]
            ## Teams who didnt won any championship has those signs       
            if(currYear !="–" and currYear !="—" and currYear !="/"):
                try:
                    if(league == "Youth-Basketball-Champions-League"): ##This leauge is not in Folder
                        currTablePath = os.getcwd()+ os.sep + league + "_" + currYear +".csv"
                    else:
                        if(league == "NBA"): ## all other leauges saved with _
                            currTablePath = os.getcwd()+ os.sep + league + os.sep + league +"-" + currYear +".csv"
                        else:
                            currTablePath = os.getcwd()+ os.sep + league + os.sep + league +"_" + currYear +".csv"
                    
                    currTable = pd.read_csv(currTablePath)
                    champInTable = process.extractOne(team, currTable["Team"]) ## Beacuse Labels was taken from other source not all the name equal to the tables
                    mask = currTable["Team"].str.contains(champInTable[0]) ## making an true/false list 
                    currTable["is_Champion"] = np.where(mask,1,0)
            
                    ###Saving the Tables with the labels
                    currTable.to_csv(currTablePath,index=False)
                    print("Saved")
                except:
                    continue
                
                        
                   
        



Eurocup
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
NBA
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Youth-Basketball-Champions-League
Saved
Australian-NBL
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Italian-Lega-Basket-Serie-A
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Bosnian-BiH-Liga
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Czech-NBL
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Ba